In [1]:
%%time
import pandas as pd


data = pd.read_csv('accepted_2007_to_2018Q4.csv',low_memory=False)
print(data.shape)

data = data.iloc[1::10]
print(data.shape)

(2260701, 151)
(226070, 151)
Wall time: 1min 18s


In [17]:
import numpy as np
def downcast_dtypes(df):
    import numpy as np
    _start = df.memory_usage(deep=True).sum() / 1024 ** 2
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [ c for c in df if df[c].dtype in ['int64','int32']]
    df[float_cols] = df[float_cols].astype(np.float16)
    df[int_cols] = df[int_cols].astype(np.int8)
    _end = df.memory_usage(deep=True).sum() / 1024 ** 2
    saved = (_start - _end) / _start * 100
    print(f"saved {saved: .2f}%")
    return df



In [18]:
data.columns

10851064

In [19]:
#missing = pd.DataFrame((data.isnull().sum() / data.shape[0] > 0.97))

In [20]:
#missing.columns = ['missing_97_percent']

In [21]:
#columns_with_97_missing = missing[missing.missing_97_percent  == True].index

In [22]:
#data = data.drop(columns_with_97_missing,axis=1)

In [23]:
#missing_90 = pd.DataFrame((data.isnull().sum() / data.shape[0] > 0.90))
#missing_90.columns = ['missing_90_percent']

In [24]:
#missing_90[missing_90.missing_90_percent == True]

In [25]:
data.loan_status.value_counts()

Fully Paid                                             107535
Current                                                 87974
Charged Off                                             26813
Late (31-120 days)                                       2162
In Grace Period                                           855
Late (16-30 days)                                         446
Does not meet the credit policy. Status:Fully Paid        195
Does not meet the credit policy. Status:Charged Off        80
Default                                                     6
Name: loan_status, dtype: int64

In [26]:
def custom_function(string):
    if string == 'Fully Paid':
        return 1
    
    if string == 'Charged Off':
        return 0
    
    if string == "Late (31-120 days)":
        return 0
    
    if string == "Late (16-30 days)":
        return 0
        
    if string == 'Does not meet the credit policy. Status:Fully Paid':
        return 1
    
    if string == 'Does not meet the credit policy. Status:Charged Off':
        return 0
    
    if string == 'Default':
        return 0
    
    else:
        return 'active'
    
    
data['loan_status'] = data.loan_status.apply(lambda x:custom_function(x))    

In [27]:
active_rows = data[data.loan_status == 'active'].index.tolist()

data = data.drop(active_rows,axis=0)

In [28]:
data = data.fillna(0)

In [29]:
missing_fractions = data.isnull().mean().sort_values(ascending=False)
drop_list = sorted(list(missing_fractions[missing_fractions > 0.3].index))
data = data.drop(labels=drop_list, axis=1, inplace=True)

In [34]:


imp_cols = ['acc_now_delinq', 'acc_open_past_24mths', 'addr_state', 'all_util', 'annual_inc', 'application_type', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'debt_settlement_flag', 'delinq_2yrs', 'delinq_amnt', 'dti', 'earliest_cr_line', 'emp_length', 'emp_title', 'fico_range_high', 'fico_range_low', 'funded_amnt', 'funded_amnt_inv', 'grade', 'hardship_flag', 'home_ownership', 'id', 'initial_list_status', 'inq_fi', 'inq_last_12m', 'inq_last_6mths', 'installment', 'int_rate', 'issue_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'last_pymnt_amnt', 'last_pymnt_d', 'loan_amnt', 'loan_status', 'max_bal_bc', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'open_acc', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'open_rv_12m', 'open_rv_24m', 'out_prncp', 'out_prncp_inv', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'policy_code', 'pub_rec', 'pub_rec_bankruptcies', 'purpose', 'pymnt_plan', 'recoveries', 'revol_bal', 'revol_util', 'sub_grade', 'tax_liens', 'term', 'title', 'tot_coll_amt', 'tot_cur_bal', 'tot_hi_cred_lim', 'total_acc', 'total_bal_ex_mort', 'total_bal_il', 'total_bc_limit', 'total_cu_tl', 'total_il_high_credit_limit', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'total_rev_hi_lim', 'url', 'verification_status', 'zip_code'].sort()

In [35]:
data = data[imp_cols]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
data = data.drop(['id','url','emp_title','title','earliest_cr_line','zip_code'],axis=1)

In [ ]:
k = []
for i in data.columns:
    if data[i].dtype == 'object':
        k.append(i)
        


for i in k:
    print(i,data[i].nunique())

In [ ]:
y = data.loan_status
x = data.drop('loan_status',axis=1)

from sklearn.preprocessing import LabelEncoder

for i in x.columns:
    x[i] = LabelEncoder().fit_transform(x[i].astype(str))

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [ ]:
%%time
from sklearn.feature_selection import RFE
from xgboost import XGBRFClassifier


model = XGBRFClassifier(n_estimators=100,use_label_encoder=False,eval_metric='error')

rfe = RFE(model,n_features_to_select=10,verbose=2)

rfe.fit(x_train,y_train)

In [ ]:
rfe.support_

In [ ]:
supported_list = [j for i,j in zip(rfe.support_,x.columns.tolist()) if i == True]

In [ ]:
supported_list
    

In [ ]:
from sklearn.metrics import f1_score,accuracy_score
from xgboost import XGBRFClassifier

tree = XGBRFClassifier(n_estimators=100,use_label_encoder=False,eval_metric='error')


tree.fit(x_train,y_train)

pred = tree.predict(x_test)

accuracy_score(pred,y_test)

In [ ]:
f1_score(pred,y_test)